# agent chat

In [1]:
from litellm import completion

import warnings
from dotenv import load_dotenv

load_dotenv("plash.env")
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

## Play With LLMs in Interactive Environments

```
./dh ipython
```


```
./dh jupyter
```

## LLMs Are Not Deterministic

In [2]:
# | echo: true
messages = [{"role": "user", "content": "What is the weather in Boston!"}]
resp = completion(model="gpt-5.2", messages=messages, caching=False)
print(resp.choices[0].message.content)

I can’t see live weather data in real time from here. If you tell me either:

- your ZIP code / neighborhood in Boston, **and**
- whether you want **right now**, **today**, or a **multi-day forecast**

…I can help you quickly interpret it, or you can share a screenshot/link from a weather app and I’ll summarize it.


In [3]:
# | echo: true
resp = completion(model="gpt-5.2", messages=messages, caching=False)
print(resp.choices[0].message.content)

I can’t see live weather data in real time from here. If you tell me **when** you mean (right now, today, this weekend) and your **zip code or neighborhood**, I can summarize what to expect and what to wear.

Fastest way to check immediately:
- National Weather Service (Boston): https://forecast.weather.gov/zipcity.php?inputstring=Boston,MA  
- Your phone’s weather app (search “Boston, MA”)

If you want, paste what you see (temp, wind, precipitation chance), and I’ll interpret it for you.


## LLM Doesn't Know The Weather Anyway ...

- give them tools

In [4]:
get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current weather for a city",
        "parameters": {
            "type": "object",
            "properties": {"city": {"type": "string", "description": "The city name"}},
            "required": ["city"],
        },
    },
}

In [5]:
# | echo: true
messages = [{"role": "user", "content": "Pick a city at random. Then get the weather for that city."}]
model = "claude-opus-4-5-20251101"
resp = completion(model, messages=messages, tools=[get_weather_tool])
print(resp.choices[0].message.content)
print("-" * 100)
resp.choices[0].message.tool_calls[0].model_dump()

I'll pick a random city - let's go with **Tokyo, Japan**.

Now let me get the weather for Tokyo:
----------------------------------------------------------------------------------------------------


{'index': 1,
 'function': {'arguments': '{"city": "Tokyo"}', 'name': 'get_weather'},
 'id': 'toolu_01SNg7MuVBAdQUgZ8BM4Y77S',
 'type': 'function'}

## Agent = LLM + Tools + Loop

:::: {.columns}
::: {.column width="40%"}
![](https://drchrislevy.com/blog/static_blog_imgs/just_loops_meme.jpeg)
:::

::: {.column width="60%"}
```python
while True:
    resp = llm(messages, tools)
    
    if not resp.tool_calls:
        return resp  # Done!
    
    run_tools(resp.tool_calls)
```
:::
::::

## Agent = LLM + Tools + Loop {.smaller}

```python
while True:
    # Call the LLM
    response = litellm.completion(
        model="claude-opus-4-5-20251101",  # "gemini/gemini-3-flash-preview", #claude-opus-4-5-20251101, gpt-5.2
        messages=messages,
        tools=TOOLS,
        reasoning_effort="low",
    )

    # Append assistant message (thought signatures automatically preserved)
    message = response.choices[0].message

    # If no tool calls, we're done - yield final response
    if not message.tool_calls:
        final_msg = {"role": "assistant", "content": message.content}
        messages.append(final_msg)
        yield final_msg
        return

    # Assistant message with tool calls - add and yield it
    messages.append(message)
    yield message

    # Execute each tool and yield results
    for tool_call in message.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        result = TOOL_FUNCTIONS[name](**args)

        tool_msg = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result,
        }
        messages.append(tool_msg)
        yield tool_msg
````

## Libraries can simplify the code

```python
@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Friendly Agent",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)


agent.run("What's the weather in Halifax?")
```

- but that's not where your going to be spending a lot of your time 
- it will be spent creating tools, wrangling data, looking at data, and making evals, and looking at more data ...

## Look at Your Data

- look at your data
- look at your data
- look at your data
- look at your data
- look at traces
- make simple tools to look at data

## Reminders

- tokens and context
- message history grows, management, keeps sending,
    - tools take up the context
- even if a library/framework makes it look like its managing message history for you, the LLM still needs to be passed the entire context with each request
- prompt caching
- reasoning - reasoning models - need to pass thinking tokens around too

## Resources

- https://hamel.dev/blog/posts/evals-faq/
- https://hamel.dev/notes/llm/evals/flashcards/
- https://docs.litellm.ai/
- https://platform.claude.com/docs/en/agents-and-tools/tool-use/overview
- https://www.anthropic.com/engineering/advanced-tool-use


## Random

- why coding agent
- why data csv
- preload data on sandbox
- increase cpu/memory on sandbox
- create sandbox async before chat begins...
- evals 
- adding back images in tool results.

## junk...

Things to remember to talk about
- prompt caching
- function calling
- reasonign models - thought tokens - need to be passed back - thinking traces need to be passed back
- LLM agent abstractions - less hackable?
- litellm
- advanced shit - https://docs.litellm.ai/blog/anthropic_advanced_features#tool-search
- https://www.anthropic.com/engineering/advanced-tool-use
- dont want a lot of tools
- progressive disclosure - tool search 
- https://docs.litellm.ai/blog/gemini_3_flash
- https://docs.litellm.ai/blog/anthropic_advanced_features
- https://github.com/openai/openai-agents-python
- context - tools blow it up
- they all have diff syntax
- Anthropic new beta runner ---> https://platform.claude.com/docs/en/agents-and-tools/tool-use/implement-tool-use

# Images in Tool Results..

In [6]:
from litellm import completion
import httpx
import base64
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

load_dotenv("plash.env")


def test_litellm(model):
    # First call - ask model to use the tool
    tool_schema = {
        "type": "function",
        "function": {"name": "get_image", "description": "Returns a test image", "parameters": {"type": "object", "properties": {}, "required": []}},
    }
    msgs = [{"role": "user", "content": "Please call the get_image tool and describe the image it returns."}]
    r = completion(model, msgs, tools=[tool_schema])

    # Get the tool call info
    tc = r.choices[0].message.tool_calls[0]
    msgs.append(r.choices[0].message)

    # Download and encode image
    img_data = httpx.get("https://raw.githubusercontent.com/AnswerDotAI/lisette/refs/heads/main/nbs/samples/puppy.jpg").content
    encoded = base64.b64encode(img_data).decode("utf-8")

    # Tool result with image
    msgs.append({"role": "tool", "tool_call_id": tc.id, "content": [{"type": "image_url", "image_url": f"data:image/jpeg;base64,{encoded}"}]})

    # Second call with image result
    r2 = completion(model, msgs, tools=[tool_schema])
    print(r2.choices[0].message.content)

In [10]:
test_litellm("gemini/gemini-3-flash-preview")  # works

0:00:01.321
This image shows a Cavalier King Charles Spaniel puppy lying on a patch of grass. The puppy has a distinctive white and chestnut (Blenheim) coat, with long, floppy chestnut ears and large, dark, expressive eyes. 

To the left of the puppy is a cluster of small, light purple daisy-like flowers. The puppy is positioned as if it is peeking out from behind the flowers, with its front paws stretched out forward. The lighting is soft and natural, suggesting an outdoor setting.


In [8]:
test_litellm("claude-opus-4-5-20251101")  # works

The image shows an adorable Cavalier King Charles Spaniel puppy lying on green grass. The puppy has the classic Blenheim coloring - a white coat with chestnut/reddish-brown markings on its ears and around its eyes. The puppy has a sweet, gentle expression with big dark eyes and a black nose.

In the background, there's a beautiful cluster of purple/lavender aster flowers that creates a lovely contrast with the puppy's coloring. The setting appears to be a garden, and the lighting suggests it was taken on a pleasant day. The puppy looks relaxed and comfortable in its pose, with its front paws stretched out in front of it.

It's a charming, heartwarming photo that captures the innocent and endearing nature of a young puppy.


In [9]:
test_litellm("gpt-5.2")  # raises error


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: OpenAIException - Invalid type for 'messages[2].content[0].image_url': expected an object, but got a string instead.

In [ ]:
from litellm import completion
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")
load_dotenv("plash.env")


def test_litellm(model):
    tool_schema = {
        "type": "function",
        "function": {
            "name": "get_image",
            "description": "Returns a test image",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    }

    msgs = [{"role": "user", "content": "Please call the get_image tool and describe the image it returns."}]

    r = completion(model, msgs, tools=[tool_schema])

    tc = r.choices[0].message.tool_calls[0]
    msgs.append(r.choices[0].message)

    img_data = httpx.get("https://raw.githubusercontent.com/AnswerDotAI/lisette/refs/heads/main/nbs/samples/puppy.jpg").content
    encoded = base64.b64encode(img_data).decode("utf-8")

    # 1) Tool outputs must be plain text/JSON for OpenAI Chat Completions
    msgs.append({"role": "tool", "tool_call_id": tc.id, "content": "Fetched image successfully. Image will be provided in the next user message."})

    # 2) Provide the image as a *user* multimodal message
    # NOTE: OpenAI expects image_url to be an object: {"url": "..."}
    msgs.append(
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Here is the image returned by get_image. Please describe it."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded}"}},
            ],
        }
    )

    r2 = completion(model, msgs, tools=[tool_schema])
    print(r2.choices[0].message.content)

In [ ]:
test_litellm("gemini/gemini-3-flash-preview")  # should still work

In [ ]:
test_litellm("claude-opus-4-5-20251101")  # should still work

In [ ]:
test_litellm("openai/gpt-5.2")  # this is the pattern OpenAI accepts